In [43]:
import pandas as pd
import random
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)

## extracción datos - imdb

In [2]:
with open ('../Fase1_datos/lista_drama.json', 'r') as fichero:
    drama = json.load(fichero)

In [11]:
len(drama)

5210

In [29]:
busqueda = drama[1801:2801]

In [30]:
len(busqueda)

1000

In [12]:
fallos = []
info_por_peli = []

In [31]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))


# variables donde iremos almacenando todo
#info_por_peli = []


# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]

    try:
    # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
            
    except:
        fallos.append(peli)




## limpieza datos - imdb


In [44]:
with open('InfoDrama.json', 'r') as fichero:
    info_por_peli = json.load(fichero)

In [47]:
info_por_peli

[['tt0113092',
  ['Reparto y equipo',
   'Reseñas de usuarios',
   'Curiosidades',
   'IMDbPro',
   'Todos los temas',
   'Por la causa',
   'Título original: For the Cause',
   '2000',
   'R',
   '1h 40min',
   'PUNTUACIÓN EN IMDb',
   '3,4',
   '/10',
   '841',
   'TU PUNTUACIÓN',
   'Puntuar',
   'Acción',
   'Aventura',
   'Drama',
   'Añade un argumento en tu idioma',
   'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?',
   'Dirección',
   'David DouglasTim Douglas',
   'Guión',
   'David DouglasChristopher Salazar',
   'Reparto principal',
   'Dean CainThomas Ian GriffithJustin Whalin',
   'Ver la información de la producción en IMDbPro',
   'Añade a mi lista de seguimiento',
   '9',
   'Reseñas de usuarios',
  

In [54]:
lista_tuplas = []

for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    tupla_datos = [peli[0],] 
    # listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])
    
   
    # añadimos a la tupla los datos de la lista intermedia
    if direccion != []:
        tupla_datos.append(direccion[0])
    else:
        tupla_datos.append(np.nan)
    

    if guion != []:
        tupla_datos.append(guion[0])
    else:
        tupla_datos.append(np.nan)
    

    if nota != []:
        tupla_datos.append(nota[0])
    else:
        tupla_datos.append(np.nan)
    

    if duracion != []:
        tupla_datos.append(duracion[0])
    else:
        tupla_datos.append(np.nan)
    

    if argumento != []:
        tupla_datos.append(argumento[0])
    else:
        tupla_datos.append(np.nan)
    
    # añadimos a la lista general
    lista_tuplas.append(tuple(tupla_datos))

In [55]:
tupla_datos

['tt0383929',
 'Samm Styles',
 nan,
 6.4,
 '1h 56min',
 "Inmate activist George Lester Jackson's short life became a flashpoint for revolution, igniting the bloodiest riot in San Quentin's history."]

In [59]:
lista_tuplas

[('tt0113092',
  'David Douglas, Tim Douglas',
  'David Douglas, Christopher Salazar',
  3.4,
  '1h 40min',
  'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?'),
 ('tt0113086',
  'Michael New',
  nan,
  6.8,
  '2000',
  "Florentino is a young horseman and singer, a free spirit living in the open world without restrictions, in the Venezuelan plains. He confronts the devil in a duel of improvised verses. As time passes, these unspoilt plains start to change. Through Florentino's voice and actions, the plains cannot be conquered. Its culture defies the devil in the eternal fight between ... Leer todo"),
 ('tt0116748',
  'Rakesh Roshan',
  'Sachin Bhowmick, Ravi Kapoor, Sagar Sarhadi',
  3.7,
  '3h',
  "Wealthy Rajiv Sinh

In [64]:
with open('MuestraDrama.json', 'w') as fichero:
    json.dump(lista_tuplas, fichero)
        